<a href="https://colab.research.google.com/github/AbdelrahmanElsherif/NeuralNetwork-Based-Generic-Compression/blob/master/Audio/Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%cd ./drive/My Drive/audio_compression

/content/drive/My Drive/audio_compression


In [0]:
!ls

audio.rar	    autoencoder3.model	     autoencoder.model
audio_wav	    autoencoder4.model	     autoencoder_samaa.model
audio_wav_training  autoencoder5.model	     cp.ckpt
autoencoder2.model  autoencoder_dense.model  trial_train_data.csv


In [0]:
import wave, struct
import keras
from keras.models import Model
from keras.layers import Dense, Input, LSTM
import numpy as np
import os

Using TensorFlow backend.


In [0]:
in_layer = Input(shape=(441,))
# Construct the encoder layers
encode = Dense(416, activation='relu')(in_layer)
encode = Dense(416 // 2, activation='relu')(encode)
encode = Dense(416 // 4, activation='relu')(encode)
encode = Dense(416 // 8, activation='relu')(encode)

# Construct the decoder layers
decode = Dense(416 // 4, activation='relu')(encode)
decode = Dense(416 // 2, activation='relu')(decode)
decode = Dense(416, activation='sigmoid')(decode)

# The autoencoder is the whole thing
autoencoder = Model(in_layer, decode)

# Compile the model
autoencoder.compile('adam', loss='mean_squared_logarithmic_error')

In [0]:
cp_callback = keras.callbacks.ModelCheckpoint(filepath="cp.ckpt",
                                                 save_weights_only=True,
                                                 verbose=1)

In [0]:
def dataFromWave(fname):
    """
    Reads a wav file to samples
    """
    f = wave.open(fname, 'rb')
    # Read Channel Number
    chans = f.getnchannels()
    # Get raw sample count
    samps = f.getnframes()
    # Get bit-width of samples
    sampwidth = f.getsampwidth()
    # Get sampling rate
    rate = f.getframerate()
    # Read samples
    if sampwidth == 3:  # have to read this one sample at a time
        s = ''
        for k in range(samps):
            fr = f.readframes(1)
            for c in range(0, 3 * chans, 3):
                s += '\0' + fr[c:(c + 3)]  # put TRAILING 0 to make 32-bit (file is little-endian)
    else:
        s = f.readframes(samps)
    f.close()
    # Unpack samples
    unpstr = '<{0}{1}'.format(samps * chans, {1: 'b', 2: 'h', 3: 'i', 4: 'i', 8: 'q'}[sampwidth])
    x = list(struct.unpack(unpstr, s))
    if sampwidth == 3:
        x = [k >> 8 for k in x]  # downshift to get +/- 2^24 with sign extension

    return x, chans, samps, sampwidth, rate

In [0]:
def train(train_data):
    train_data = train_data.reshape((rows, 441))
    autoencoder.fit(train_data, train_data, epochs=5, shuffle=True, callbacks=[cp_callback])
    autoencoder.save("autoencoder_new.model")

In [0]:
DATA_FILES_WAV = "audio_wav_training"
def load_data():
    train_data = np.array([])
    rows = 0
    directory = os.fsencode(DATA_FILES_WAV)
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        print(filename)
        if filename.endswith(".wav"):
            data, chans, samps, width, samp_rate = dataFromWave(DATA_FILES_WAV+"/"+filename)
            rate = samp_rate // 100
            print("rate=", rate)
            data = np.array(data)
            data = data.astype(float) / float(pow(2, 15))
            data += 1.0
            data = data / 2.0
            print('data.shpae=', data.shape)
            n_in = len(data)
            p_size = n_in + (rate - (n_in % rate))
            print('p_size=', p_size)
            padded = np.zeros((p_size,))
            print('padded.shape=', padded.shape)
            padded[0:n_in] = data
            print('padded.shape=', padded.shape)
            inputs = padded.reshape((len(padded) // rate, rate))
            print('inputs.shape=', inputs.shape)
            rows += len(padded) // rate
            train_data = np.append(train_data, inputs)
    return train_data, rows

In [0]:
train_data, rows = load_data()

0.wav
rate= 441
data.shpae= (10584000,)
p_size= 10584441
padded.shape= (10584441,)
padded.shape= (10584441,)
inputs.shape= (24001, 441)
1.wav
rate= 441
data.shpae= (10584000,)
p_size= 10584441
padded.shape= (10584441,)
padded.shape= (10584441,)
inputs.shape= (24001, 441)
2.wav
rate= 441
data.shpae= (10584000,)
p_size= 10584441
padded.shape= (10584441,)
padded.shape= (10584441,)
inputs.shape= (24001, 441)
3.wav
rate= 441
data.shpae= (10584000,)
p_size= 10584441
padded.shape= (10584441,)
padded.shape= (10584441,)
inputs.shape= (24001, 441)
4.wav
rate= 441
data.shpae= (10584000,)
p_size= 10584441
padded.shape= (10584441,)
padded.shape= (10584441,)
inputs.shape= (24001, 441)
5.wav
rate= 441
data.shpae= (10584000,)
p_size= 10584441
padded.shape= (10584441,)
padded.shape= (10584441,)
inputs.shape= (24001, 441)
6.wav
rate= 441
data.shpae= (10584000,)
p_size= 10584441
padded.shape= (10584441,)
padded.shape= (10584441,)
inputs.shape= (24001, 441)
7.wav
rate= 441
data.shpae= (10584000,)
p_size= 

In [0]:
train(train_data) #loss = 'mean_squared_logarithmic_error'

Epoch 1/10
600025/600025 [==============================] - 119s 198us/step - loss: 0.0012

Epoch 00001: saving model to cp2.ckpt
Epoch 2/10
600025/600025 [==============================] - 118s 197us/step - loss: 9.1214e-04

Epoch 00002: saving model to cp2.ckpt
Epoch 3/10
600025/600025 [==============================] - 119s 198us/step - loss: 8.6840e-04

Epoch 00003: saving model to cp2.ckpt
Epoch 4/10
600025/600025 [==============================] - 117s 194us/step - loss: 8.4963e-04

Epoch 00004: saving model to cp2.ckpt
Epoch 5/10
600025/600025 [==============================] - 116s 194us/step - loss: 8.3033e-04

Epoch 00005: saving model to cp2.ckpt
Epoch 6/10
600025/600025 [==============================] - 117s 194us/step - loss: 8.2279e-04

Epoch 00006: saving model to cp2.ckpt
Epoch 7/10
600025/600025 [==============================] - 116s 193us/step - loss: 8.0564e-04

Epoch 00007: saving model to cp2.ckpt
Epoch 8/10
600025/600025 [==============================] - 117s 1

In [0]:
train(train_data) #loss = 'categorical_hinge'

Epoch 1/10
600025/600025 [==============================] - 66s 110us/step - loss: 6.0191e-05

Epoch 00001: saving model to cp2.ckpt
Epoch 2/10
600025/600025 [==============================] - 65s 108us/step - loss: 5.5439e-05

Epoch 00002: saving model to cp2.ckpt
Epoch 3/10
600025/600025 [==============================] - 64s 107us/step - loss: 5.1483e-05

Epoch 00003: saving model to cp2.ckpt
Epoch 4/10
600025/600025 [==============================] - 64s 107us/step - loss: 4.8544e-05

Epoch 00004: saving model to cp2.ckpt
Epoch 5/10
600025/600025 [==============================] - 64s 107us/step - loss: 4.6407e-05

Epoch 00005: saving model to cp2.ckpt
Epoch 6/10
600025/600025 [==============================] - 65s 108us/step - loss: 4.4958e-05

Epoch 00006: saving model to cp2.ckpt
Epoch 7/10
600025/600025 [==============================] - 65s 108us/step - loss: 4.4028e-05

Epoch 00007: saving model to cp2.ckpt
Epoch 8/10
600025/600025 [==============================] - 66s 110us

In [0]:
train(train_data) #loss = 'logcosh'

Epoch 1/10
600025/600025 [==============================] - 62s 103us/step - loss: 0.0013

Epoch 00001: saving model to cp2.ckpt
Epoch 2/10
600025/600025 [==============================] - 62s 103us/step - loss: 9.2997e-04

Epoch 00002: saving model to cp2.ckpt
Epoch 3/10
600025/600025 [==============================] - 62s 104us/step - loss: 8.5214e-04

Epoch 00003: saving model to cp2.ckpt
Epoch 4/10
600025/600025 [==============================] - 62s 103us/step - loss: 8.1542e-04

Epoch 00004: saving model to cp2.ckpt
Epoch 5/10
600025/600025 [==============================] - 62s 104us/step - loss: 7.9837e-04

Epoch 00005: saving model to cp2.ckpt
Epoch 6/10
600025/600025 [==============================] - 63s 104us/step - loss: 7.8138e-04

Epoch 00006: saving model to cp2.ckpt
Epoch 7/10
600025/600025 [==============================] - 63s 105us/step - loss: 7.7484e-04

Epoch 00007: saving model to cp2.ckpt
Epoch 8/10
600025/600025 [==============================] - 63s 105us/ste

In [0]:
train(train_data) #loss = 'huber_loss'

Epoch 1/10
600025/600025 [==============================] - 62s 103us/step - loss: 0.0013

Epoch 00001: saving model to cp2.ckpt
Epoch 2/10
600025/600025 [==============================] - 62s 103us/step - loss: 9.1942e-04

Epoch 00002: saving model to cp2.ckpt
Epoch 3/10
600025/600025 [==============================] - 61s 102us/step - loss: 8.4184e-04

Epoch 00003: saving model to cp2.ckpt
Epoch 4/10
600025/600025 [==============================] - 62s 104us/step - loss: 8.0253e-04

Epoch 00004: saving model to cp2.ckpt
Epoch 5/10
600025/600025 [==============================] - 61s 102us/step - loss: 7.7631e-04

Epoch 00005: saving model to cp2.ckpt
Epoch 6/10
600025/600025 [==============================] - 61s 102us/step - loss: 7.6469e-04

Epoch 00006: saving model to cp2.ckpt
Epoch 7/10
600025/600025 [==============================] - 61s 102us/step - loss: 7.5884e-04

Epoch 00007: saving model to cp2.ckpt
Epoch 8/10
600025/600025 [==============================] - 61s 102us/ste

In [0]:
train(train_data) #loss = 'kullback_leibler_divergence'

Epoch 1/10
600025/600025 [==============================] - 62s 103us/step - loss: -143.5052

Epoch 00001: saving model to cp2.ckpt
Epoch 2/10
600025/600025 [==============================] - 62s 104us/step - loss: -143.5717

Epoch 00002: saving model to cp2.ckpt
Epoch 3/10
600025/600025 [==============================] - 63s 104us/step - loss: -143.5717

Epoch 00003: saving model to cp2.ckpt
Epoch 4/10
600025/600025 [==============================] - 63s 106us/step - loss: -143.5717

Epoch 00004: saving model to cp2.ckpt
Epoch 5/10
600025/600025 [==============================] - 64s 107us/step - loss: -143.5717

Epoch 00005: saving model to cp2.ckpt
Epoch 6/10
600025/600025 [==============================] - 63s 106us/step - loss: -143.5717

Epoch 00006: saving model to cp2.ckpt
Epoch 7/10
600025/600025 [==============================] - 65s 108us/step - loss: -143.5717

Epoch 00007: saving model to cp2.ckpt
Epoch 8/10
600025/600025 [==============================] - 63s 106us/step -

In [0]:
train(train_data) #loss = 'cosine_proximity'

Epoch 1/10
600025/600025 [==============================] - 66s 109us/step - loss: -0.9945

Epoch 00001: saving model to cp2.ckpt
Epoch 2/10
600025/600025 [==============================] - 66s 109us/step - loss: -0.9956

Epoch 00002: saving model to cp2.ckpt
Epoch 3/10
600025/600025 [==============================] - 65s 109us/step - loss: -0.9958

Epoch 00003: saving model to cp2.ckpt
Epoch 4/10
600025/600025 [==============================] - 66s 110us/step - loss: -0.9960

Epoch 00004: saving model to cp2.ckpt
Epoch 5/10
600025/600025 [==============================] - 66s 109us/step - loss: -0.9960

Epoch 00005: saving model to cp2.ckpt
Epoch 6/10
600025/600025 [==============================] - 66s 109us/step - loss: -0.9960

Epoch 00006: saving model to cp2.ckpt
Epoch 7/10
600025/600025 [==============================] - 65s 109us/step - loss: -0.9960

Epoch 00007: saving model to cp2.ckpt
Epoch 8/10
600025/600025 [==============================] - 65s 108us/step - loss: -0.9960

# ***The chosen one: mean_squared_logarithmic_error***

In [0]:
in_layer = Input(shape=(441,))
# Construct the encoder layers
encode = Dense(441, activation='relu')(in_layer)
encode = Dense(441 // 2, activation='relu')(encode)
encode = Dense(441 // 4, activation='relu')(encode)
encode = Dense(441 // 8, activation='relu')(encode)
encode = Dense(441 // 16, activation='relu')(encode)

# Construct the decoder layers
decode = Dense(441 // 8, activation='relu')(encode)
decode = Dense(441 // 4, activation='relu')(decode)
decode = Dense(441 // 2, activation='relu')(decode)
decode = Dense(441, activation='sigmoid')(decode)

# The autoencoder is the whole thing
autoencoder = Model(in_layer, decode)

# Compile the model
autoencoder.compile('Adamax', loss='mean_squared_logarithmic_error')

In [0]:
train(train_data) #using adamax optimizer

Epoch 1/10
600025/600025 [==============================] - 63s 105us/step - loss: 0.0015

Epoch 00001: saving model to cp2.ckpt
Epoch 2/10
600025/600025 [==============================] - 63s 105us/step - loss: 0.0011

Epoch 00002: saving model to cp2.ckpt
Epoch 3/10
600025/600025 [==============================] - 62s 104us/step - loss: 0.0010

Epoch 00003: saving model to cp2.ckpt
Epoch 4/10
600025/600025 [==============================] - 61s 102us/step - loss: 0.0010

Epoch 00004: saving model to cp2.ckpt
Epoch 5/10
600025/600025 [==============================] - 61s 102us/step - loss: 0.0010

Epoch 00005: saving model to cp2.ckpt
Epoch 6/10
600025/600025 [==============================] - 61s 102us/step - loss: 0.0010

Epoch 00006: saving model to cp2.ckpt
Epoch 7/10
600025/600025 [==============================] - 62s 104us/step - loss: 0.0010

Epoch 00007: saving model to cp2.ckpt
Epoch 8/10
600025/600025 [==============================] - 62s 103us/step - loss: 9.9787e-04

Ep

In [0]:
train(train_data) #using adam optimizer

Epoch 1/10
600025/600025 [==============================] - 75s 125us/step - loss: 0.0016

Epoch 00001: saving model to cp2.ckpt
Epoch 2/10
600025/600025 [==============================] - 74s 123us/step - loss: 0.0012

Epoch 00002: saving model to cp2.ckpt
Epoch 3/10
600025/600025 [==============================] - 72s 121us/step - loss: 0.0012

Epoch 00003: saving model to cp2.ckpt
Epoch 4/10
600025/600025 [==============================] - 72s 120us/step - loss: 0.0012

Epoch 00004: saving model to cp2.ckpt
Epoch 5/10
600025/600025 [==============================] - 73s 121us/step - loss: 0.0011

Epoch 00005: saving model to cp2.ckpt
Epoch 6/10
600025/600025 [==============================] - 72s 120us/step - loss: 0.0011

Epoch 00006: saving model to cp2.ckpt
Epoch 7/10
600025/600025 [==============================] - 71s 119us/step - loss: 0.0011

Epoch 00007: saving model to cp2.ckpt
Epoch 8/10
600025/600025 [==============================] - 72s 120us/step - loss: 0.0011

Epoch 

In [0]:
train(train_data) #training on 50 files

Epoch 1/10
1200014/1200014 [==============================] - 131s 109us/step - loss: 0.0016

Epoch 00001: saving model to cp2.ckpt
Epoch 2/10
1200014/1200014 [==============================] - 128s 107us/step - loss: 0.0013

Epoch 00002: saving model to cp2.ckpt
Epoch 3/10
1200014/1200014 [==============================] - 128s 107us/step - loss: 0.0012

Epoch 00003: saving model to cp2.ckpt
Epoch 4/10
1200014/1200014 [==============================] - 128s 107us/step - loss: 0.0012

Epoch 00004: saving model to cp2.ckpt
Epoch 5/10
1200014/1200014 [==============================] - 127s 106us/step - loss: 0.0011

Epoch 00005: saving model to cp2.ckpt
Epoch 6/10
1200014/1200014 [==============================] - 127s 106us/step - loss: 0.0011

Epoch 00006: saving model to cp2.ckpt
Epoch 7/10
1200014/1200014 [==============================] - 127s 106us/step - loss: 0.0011

Epoch 00007: saving model to cp2.ckpt
Epoch 8/10
1200014/1200014 [==============================] - 127s 106us/ste

In [0]:
 train(train_data) #trained on 100 files

Epoch 1/10
2399870/2399870 [==============================] - 327s 136us/step - loss: 0.0016

Epoch 00001: saving model to cp.ckpt
Epoch 2/10
2399870/2399870 [==============================] - 324s 135us/step - loss: 0.0014

Epoch 00002: saving model to cp.ckpt
Epoch 3/10
2399870/2399870 [==============================] - 344s 144us/step - loss: 0.0013

Epoch 00003: saving model to cp.ckpt
Epoch 4/10
2399870/2399870 [==============================] - 340s 142us/step - loss: 0.0013

Epoch 00004: saving model to cp.ckpt
Epoch 5/10
2399870/2399870 [==============================] - 334s 139us/step - loss: 0.0013

Epoch 00005: saving model to cp.ckpt
Epoch 6/10
 926176/2399870 [==========>...................] - ETA: 3:23 - loss: 0.0013

In [0]:
autoencoder.load_weights('cp.ckpt')
train(train_data) #continue training on 100 files

Epoch 1/5
2399870/2399870 [==============================] - 339s 141us/step - loss: 0.0012

Epoch 00001: saving model to cp.ckpt
Epoch 2/5
2399870/2399870 [==============================] - 334s 139us/step - loss: 0.0012

Epoch 00002: saving model to cp.ckpt
Epoch 3/5
2399870/2399870 [==============================] - 331s 138us/step - loss: 0.0012

Epoch 00003: saving model to cp.ckpt
Epoch 4/5
2399870/2399870 [==============================] - 327s 136us/step - loss: 0.0012

Epoch 00004: saving model to cp.ckpt
Epoch 5/5
2399870/2399870 [==============================] - 326s 136us/step - loss: 0.0012

Epoch 00005: saving model to cp.ckpt
